In [ ]:
from aisurveywriter.tasks import ReferenceExtractor
from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

paths = get_all_files_from_paths("../refexamples")
prompt = fh.read_yaml("../templates/prompt_config.yaml")["reference_extract_prompt"]
ref = ReferenceExtractor(llm=LLMHandler(model="gemini-2.0-flash-exp", model_type="google", temperature=0.3),
                         ref_paths=["../refexamples/ArigaK2023_ChemOfMat.pdf"], prompt=prompt,
                         raw_save_path="./raw.ref", rawbib_save_path="./rawbib.bib",
                         bib_save_path="./final.bib", cooldown_sec=50, batches=4)
ref()

In [ ]:
from aisurveywriter.core.paper import PaperData

paper = PaperData.from_tex("../out/generated-rawscratch.tex", "test")
for sec in paper.sections:
    print(sec.title)